In [ ]:
print('Lets build our first agent demo - an angel from future.')

In [ ]:
# installing all the required libraries
!pip install -U ipywidgets phidata openai python-dotenv yfinance packaging duckduckgo-search fastapi uvicorn groq python-docx python-multipart

In [ ]:
'''create a .env file in the same folder to the .ipynb file and inside the file paste your api keys. Please ensure this should not be public and you do not upload this file to any code sharing platform.

.env file content
OPENAI_API_KEY=<paste the open api key>
PHIDATA_API_KEY=<paste the phidata api key>
GROQ_API_KEY=<paste the groq api key>'''

# code
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the API_KEY value
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") -- not needed as we have used groq model for multi agent, if we do not define that then default it will pull 'gpt-4o' model from OpenAI
PHIDATA_API_KEY = os.getenv("PHIDATA_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [ ]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.duckduckgo import DuckDuckGo

In [ ]:
'''
# This is a test code to check the setup. It is not required for the Agentic AI demo.

import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Who is Sachin Tendulkar",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)'''

In [ ]:
# Agent 1 : Web Search Agent

web_search_agent = Agent(
    name="Web Search Agent",
    role="Search the web for the information",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[DuckDuckGo()],
    instructions=["Always include the sources", "Use tables to display the information"],
    show_tool_calls=True,
    markdown=True
)

In [ ]:
# Agent 2 : Financial Agent

financial_agent = Agent(
    name="Finance AI Agent",
    role="Please provide financial information",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            stock_fundamentals=True,
            company_news=True
        )
    ],
    instructions=["Use tables to display the information"],
    show_tool_calls=True,
    markdown=True
)

In [ ]:
# Forming Multi Model AI Agent

multi_ai_agents = Agent(
    team=[web_search_agent, financial_agent],
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=["Always include the sources", "Use tables to display the information"],
    show_tool_calls=True,
    markdown=True
)

In [ ]:
# multi_ai_agents.print_response("Summarize analyst recommendations and share the latest news for NVDA", stream=True)

multi_ai_agents.print_response("Summarize analyst recommendations and share the latest news for NVDA")

In [ ]:
import openai
import phi.api
from phi.model.openai import OpenAIChat
from phi.playground import Playground, serve_playground_app
import uvicorn
import nest_asyncio
import multiprocessing

In [ ]:
nest_asyncio.apply()
app = Playground(agents=[financial_agent, web_search_agent]).get_app()

uvicorn.run(app, host="0.0.0.0", port=7777)

'''def run_server():
    uvicorn.run(app, host="0.0.0.0", port=7777)

server_process = multiprocessing.Process(target=run_server)
server_process.start()'''